In [24]:
import os
import csv
import pandas as pd
import numpy as np
import math
import requests
from os import sep
from google.colab import drive
import glob
!pip install SGP4
import sgp4
from sgp4.earth_gravity import wgs72
from sgp4.api import Satrec, WGS72
from sgp4.io import twoline2rv
from sgp4.api import jday
from datetime import datetime
!pip install pyproj
import pyproj
!pip install skyfield
from skyfield.api import load

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.0/446.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 5.3 MB/s eta 0:00:00


In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
%cd drive/MyDrive/Rafel-Satellitecollision-GSoC2023/
%ls

/content/drive/.shortcut-targets-by-id/1NyJDOWUmDY1vQY1mod9dhwk8XC_AaT3-/Rafel-Satellitecollision-GSoC2023
'Anotations  TODO.gdoc'
 collision_satellites.ipynb
 creating_model_.ipynb
 data/
'[GSoC 2023 apk testing] Satellite Space Collision prevision - Rafel Salgueiro - Moiśes Martínez'/
'[GSoC 2023 worklog] Satellite Space Collision prevision - Rafel Salgueiro - Moiśes Martínez.gsheet'
 non_collision_satellites.ipynb


# Data treatment

In [4]:
%cd data/no_collision

/content/drive/.shortcut-targets-by-id/1NyJDOWUmDY1vQY1mod9dhwk8XC_AaT3-/Rafel-Satellitecollision-GSoC2023/data/no_collision


In [5]:
extension = 'csv'
all_files = [i for i in glob.glob('*.{}'.format(extension))]
print (all_files)

['non_colision_satelites.csv']


In [6]:
all_non_colision_satellites = pd.read_csv('non_colision_satelites.csv')

In [7]:
sat_filt = all_non_colision_satellites.drop_duplicates(subset = 'OBJECT_ID')
sat_filt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11856 entries, 0 to 63043
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OBJECT_NAME                      11856 non-null  object 
 1   OBJECT_ID                        11855 non-null  object 
 2   EPOCH                            11856 non-null  object 
 3   MEAN_MOTION                      11856 non-null  float64
 4   ECCENTRICITY                     11856 non-null  float64
 5   INCLINATION                      11856 non-null  float64
 6   RA_OF_ASC_NODE                   11856 non-null  float64
 7   ARG_OF_PERICENTER                11856 non-null  float64
 8   MEAN_ANOMALY                     11856 non-null  float64
 9   EPHEMERIS_TYPE                   11856 non-null  float64
 10  CLASSIFICATION_TYPE              11856 non-null  object 
 11  NORAD_CAT_ID                     11856 non-null  float64
 12  ELEMENT_SET_NO    

In [8]:
pd.set_option('display.max_rows', 10)
sat_filt['OBJECT_ID'].value_counts()

2023-063A     1
2023-058K     1
2023-058A     1
2023-058B     1
2023-058C     1
             ..
2021-038BF    1
2021-038BG    1
2021-038BH    1
2021-038BJ    1
2020-087A     1
Name: OBJECT_ID, Length: 11855, dtype: int64

In [9]:
sat_filt = sat_filt.drop(['GROUP "gpz" does not exist', 'GROUP "gpz-plus" does not exist'], axis =1)

In [10]:
sat_filt.isna()

,OBJECT_NAME,OBJECT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,CLASSIFICATION_TYPE,NORAD_CAT_ID,ELEMENT_SET_NO,REV_AT_EPOCH,BSTAR,MEAN_MOTION_DOT,MEAN_MOTION_DDOT
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19732,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19735,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19736,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19794,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [11]:
sat_filt = sat_filt.dropna()
sat_filt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11855 entries, 0 to 63043
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   OBJECT_NAME          11855 non-null  object 
 1   OBJECT_ID            11855 non-null  object 
 2   EPOCH                11855 non-null  object 
 3   MEAN_MOTION          11855 non-null  float64
 4   ECCENTRICITY         11855 non-null  float64
 5   INCLINATION          11855 non-null  float64
 6   RA_OF_ASC_NODE       11855 non-null  float64
 7   ARG_OF_PERICENTER    11855 non-null  float64
 8   MEAN_ANOMALY         11855 non-null  float64
 9   EPHEMERIS_TYPE       11855 non-null  float64
 10  CLASSIFICATION_TYPE  11855 non-null  object 
 11  NORAD_CAT_ID         11855 non-null  float64
 12  ELEMENT_SET_NO       11855 non-null  float64
 13  REV_AT_EPOCH         11855 non-null  float64
 14  BSTAR                11855 non-null  float64
 15  MEAN_MOTION_DOT      11855 non-null 

In [12]:
sat_filt = sat_filt[:-1]

<h1> Download TLE data to get the positions

In [13]:
%cd tle_sat

/content/drive/.shortcut-targets-by-id/1NyJDOWUmDY1vQY1mod9dhwk8XC_AaT3-/Rafel-Satellitecollision-GSoC2023/data/no_collision/tle_sat


In [14]:
groups = ['last-30-days', 'stations', 'visual', 'active', 'analyst', '1982-092', '1999-025', 'iridium-33-debris', 'cosmos-2251-debris', 'weather', 'noaa', 'goes', 'resource', 'sarsat', 'tdrss', 'argos', 'spire', 'planet', 'dmc', 'ses', 'iridium-NEXT', 'oneweb', 'globalstar', 'amateur', 'other-comm', 'intelsat', 'iridium', 'starlink', 'orbcomm', 'swarm', 'x-comm', 'gorizont', 'satnogs', 'raduga', 'molniya', 'geo', 'gnss', 'gps-ops', 'galileo', 'glo-ops', 'beidou', 'sbas', 'nnss', 'musson', 'science', 'geodetic', 'education', 'engineering', 'radar', 'other', 'military', 'cubesat']

In [15]:
tle_data = ''
for group in groups:
  url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=' + group + '&FORMAT=tle'
  response = requests.get(url)
  tle_data += response.text


In [16]:
tle_lines = tle_data.strip().split('\n')

In [17]:
with open('tle_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)

        # Escribir el encabezado del archivo CSV
        writer.writerow(['satellite_name', 'tle_line1', 'tle_line2'])

        # Escribir los datos TLE en el archivo CSV
        for i in range(0, len(tle_lines), 3):
            satellite_name = tle_lines[i].strip()
            tle_line1 = tle_lines[i+1].strip()
            tle_line2 = tle_lines[i+2].strip()
            writer.writerow([satellite_name, tle_line1, tle_line2])

In [18]:
tle_dataframe = pd.read_csv('tle_data.csv')

In [19]:
tle_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20224 entries, 0 to 20223
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   satellite_name  20224 non-null  object
 1   tle_line1       20224 non-null  object
 2   tle_line2       20224 non-null  object
dtypes: object(3)
memory usage: 474.1+ KB


In [20]:
tle_df = tle_dataframe.drop_duplicates(subset = 'tle_line1')

In [21]:
pd.set_option('display.max_rows', 10)
tle_df['satellite_name'].value_counts()

FENGYUN 1C DEB     2273
COSMOS 2251 DEB     853
UNKNOWN             326
IRIDIUM 33 DEB      206
COSMOS 1408 DEB     105
                   ... 
STARLINK-1680         1
STARLINK-1679         1
STARLINK-1676         1
STARLINK-1671         1
COSMOS 2454 [-]       1
Name: satellite_name, Length: 8545, dtype: int64

Position with sgp4 library

In [25]:
hours = range(12)
load_data = load('de421.bsp')
earth = load_data['earth']
position_data = []

for _, row in sat_filt.iterrows():
    object_name = row['OBJECT_NAME']

    tle_row = tle_df.loc[tle_df['satellite_name'] == object_name]

    if not tle_row.empty:
        line1 = tle_row['tle_line1'].values[0]
        line2 = tle_row['tle_line2'].values[0]

        satellite = Satrec.twoline2rv(line1, line2)
        position_data_row = []

        for hour in hours:
            datetime_propagate = datetime(year=2023, month=6, day=16, hour=hour)

            jd, fr = jday(datetime_propagate.year, datetime_propagate.month, datetime_propagate.day, datetime_propagate.hour, 0, 0)

            _, position, _ = satellite.sgp4(jd, fr)

            position_data_row.append(position)

    position_data.append(position_data_row)

for i, hour in enumerate(hours):
    sat_filt['POSITION ' + str(hour)] = [position[i] if len(position) > i else None for position in position_data]


In [26]:
sat_filt.head(30)

,OBJECT_NAME,OBJECT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,...,POSITION 2,POSITION 3,POSITION 4,POSITION 5,POSITION 6,POSITION 7,POSITION 8,POSITION 9,POSITION 10,POSITION 11
0,TIANZHOU-6,2023-063A,2023-06-08T14:04:25.588128,15.598920,0.000315,41.4732,208.1454,316.1164,43.9424,0.0,...,"(-5821.652669321065, -1710.0984623873692, 2980...","(857.9491712343827, 5025.544255785977, -4459.5...","(4823.261254368265, -4194.54460935473, 2213.67...","(-6505.3723664759655, -62.83594972367779, 1845...","(2837.5829635476257, 4301.279718158983, -4396....","(3177.9839535684105, -4989.486638854572, 3275....","(-6550.556544614732, 1600.0456667255362, 541.3...","(4534.587617202759, 3137.5776575710443, -3934....","(1228.2284355749393, -5280.763401508638, 4037....","(-5955.312681867662, 3109.542837284911, -812.6..."
1,STARLINK-5990,2023-064A,2023-06-08T09:21:43.733664,15.696619,0.000375,69.9973,164.7275,281.5263,78.5511,0.0,...,"(2726.9188073088235, 682.6694178165881, -5969....","(2960.395594737866, -4123.914275217444, 4212.6...","(-5425.253280577058, 3111.6731545364414, 2096....","(2044.767894113304, 1264.279993810242, -6149.0...","(3536.6491535415616, -4294.971357564083, 3553....","(-5295.1192936586895, 2711.84536638728, 2856.9...","(1385.0864981231116, 1786.9374382667058, -6206...","(4006.5079780633864, -4389.247044476826, 2885....","(-5095.38596469003, 2301.4560701526534, 3512.8...","(764.6425205667464, 2247.797132340576, -6168.3..."
2,STARLINK-5993,2023-064B,2023-06-08T07:50:34.638720,15.696613,0.000454,69.9975,164.9054,274.6979,85.3699,0.0,...,"(-5470.765726604093, 3266.3492279016564, 1760....","(2391.9758660697867, 982.5401970011424, -6089....","(3208.8653617456066, -4209.64368072347, 3966.1...","(-5409.265240656421, 3007.214057011013, 2331.8...","(1930.5859371538654, 1363.8291495870137, -6182...","(3571.1512455294214, -4319.290004317599, 3520....","(-5312.378153292914, 2753.94747358425, 2824.74...","(1510.089776112283, 1697.5807203495053, -6220....","(3864.875187078024, -4393.673429027287, 3101.4...","(-5194.773131213349, 2515.610574421331, 3242.7..."
3,STARLINK-5970,2023-064C,2023-06-08T22:00:00.999936,15.681749,0.000353,69.9998,163.2467,267.3415,187.1295,0.0,...,"(2669.783166046151, 733.799428835497, -5989.02...","(3016.463188292141, -4140.033595684805, 4156.0...","(-5418.026636683286, 3072.30989994197, 2170.63...","(1977.5419491256825, 1319.3466936067587, -6158...","(3592.286464921056, -4305.21627834236, 3483.90...","(-5277.309196614738, 2663.047828904685, 2934.1...","(1308.5877945221796, 1844.8175647124058, -6206...","(4060.155523652013, -4392.592006962977, 2803.6...","(-5066.438495321995, 2243.388754478499, 3590.8...","(679.813796317047, 2307.4283452704376, -6155.6..."
4,STARLINK-5985,2023-064D,2023-06-08T09:22:12.294912,15.696452,0.000366,69.9979,164.7265,284.6637,75.4152,0.0,...,"(1220.4072777565154, 1907.2728082300005, -6201...","(4164.924580139225, -4343.85019129562, 2716.82...","(-5053.134765091664, 2112.0629883273623, 3681....","(536.6410638865966, 2392.3174919838348, -6133....","(4553.215637179648, -4352.667190081893, 1995.4...","(-4757.600263732558, 1660.2066763296918, 4266....","(-93.32139984486335, 2810.3304903127555, -5981...","(4844.26021806715, -4306.545976230865, 1306.07...","(-4429.396750506327, 1224.03475264718, 4745.67...","(-661.2380808762528, 3164.419756725041, -5768...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,STARLINK-5038,2023-064AA,2023-06-08T10:50:05.959968,15.699244,0.000331,69.9994,164.5365,262.5433,97.5386,0.0,...,"(428.626348637898, 2520.8930880312523, -6237.7...","(4318.817702657551, -4458.461706177445, 2621.5...","(-5322.646447013897, 2554.1874033675404, 3242....","(1742.5944523312949, 1567.3857473226722, -6321...","(3336.913799025333, -4348.485449634734, 3916.0...","(-5519.966989839244, 3387.08089165799, 1857.07...","(2944.769275620019, 508.9163162763645, -6043.4...","(2166.8532135935716,

In [28]:
all_data_in_one = pd.merge(sat_filt, tle_dataframe,  left_on='OBJECT_NAME', right_on='satellite_name')

In [39]:
all_data_in_one.isna()

,OBJECT_NAME,OBJECT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,EPHEMERIS_TYPE,...,POSITION 4,POSITION 5,POSITION 6,POSITION 7,POSITION 8,POSITION 9,POSITION 10,POSITION 11,tle_line1,tle_line2
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5969926,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5969927,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5969928,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5969929,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [41]:
all_data_in_one["OBJECT_NAME"].value_counts()

FENGYUN 1C DEB        2270
COSMOS 2251 DEB        854
IRIDIUM 33 DEB         210
COSMOS 1408 DEB        122
UNKNOWN                 27
                      ... 
HJ-2A                    1
HAIYANG-2C               1
JILIN-01 GAOFEN 3J       1
JILIN-01 GAOFEN 3I       1
COSMOS 2454 [-]          1
Name: OBJECT_NAME, Length: 8187, dtype: int64

In [36]:
all_data_in_one = all_data_in_one.drop_duplicates(subset = ['OBJECT_ID'])

In [30]:
all_data_in_one = all_data_in_one.drop(['satellite_name'], axis =1)

In [40]:
all_data_in_one = all_data_in_one.head(63043)

In [42]:
all_data_in_one.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11764 entries, 0 to 5969930
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   OBJECT_NAME          11764 non-null  object 
 1   OBJECT_ID            11764 non-null  object 
 2   EPOCH                11764 non-null  object 
 3   MEAN_MOTION          11764 non-null  float64
 4   ECCENTRICITY         11764 non-null  float64
 5   INCLINATION          11764 non-null  float64
 6   RA_OF_ASC_NODE       11764 non-null  float64
 7   ARG_OF_PERICENTER    11764 non-null  float64
 8   MEAN_ANOMALY         11764 non-null  float64
 9   EPHEMERIS_TYPE       11764 non-null  float64
 10  CLASSIFICATION_TYPE  11764 non-null  object 
 11  NORAD_CAT_ID         11764 non-null  float64
 12  ELEMENT_SET_NO       11764 non-null  float64
 13  REV_AT_EPOCH         11764 non-null  float64
 14  BSTAR                11764 non-null  float64
 15  MEAN_MOTION_DOT      11764 non-nul

In [46]:
all_data_in_one.head

<bound method NDFrame.head of              OBJECT_NAME  OBJECT_ID                       EPOCH  MEAN_MOTION  \
0             TIANZHOU-6  2023-063A  2023-06-08T14:04:25.588128    15.598920   
2          STARLINK-5990  2023-064A  2023-06-08T09:21:43.733664    15.696619   
4          STARLINK-5993  2023-064B  2023-06-08T07:50:34.638720    15.696613   
6          STARLINK-5970  2023-064C  2023-06-08T22:00:00.999936    15.681749   
8          STARLINK-5985  2023-064D  2023-06-08T09:22:12.294912    15.696452   
...                  ...        ...                         ...          ...   
5969926  COSMOS 2378 [-]  2001-023A  2023-06-08T14:04:57.486144    13.740027   
5969927  COSMOS 2389 [-]  2002-026A  2023-06-08T10:05:36.372768    13.750140   
5969928  COSMOS 2398 [-]  2003-023A  2023-06-08T13:20:44.105856    13.722477   
5969929  COSMOS 2429 [-]  2007-038A  2023-06-08T15:39:46.351584    13.750005   
5969930  COSMOS 2454 [-]  2009-039A  2023-06-08T11:58:38.784000    13.897359   

         

In [43]:
all_data_in_one.to_csv('all_data_sat_no_colision.csv')